<a href="https://colab.research.google.com/github/mansine/Electricity-Consumption/blob/main/PredictionByLstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)

In [33]:
df=pd.read_csv("/content/ElectricityConsumptionSum.csv")


In [34]:
df.drop(columns= 'hours',inplace=True)

In [35]:
#reading the dataset
ElectricityConsumption_data = df['ElectricityConsumption'].values
# normalize the dataset
ElectricityConsumption_data = ElectricityConsumption_data.reshape((-1,1))
# split into train and test sets
split_percent = 0.80
split = int(split_percent*len(ElectricityConsumption_data))

ElectricityConsumption_train = ElectricityConsumption_data[:split]
ElectricityConsumption_test = ElectricityConsumption_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]
print(len(ElectricityConsumption_train))
print(len(ElectricityConsumption_test))


20580
5146


In [36]:
from keras.preprocessing.sequence import TimeseriesGenerator

look_back = 15

train_generator = TimeseriesGenerator(ElectricityConsumption_train,ElectricityConsumption_train,length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(ElectricityConsumption_test, ElectricityConsumption_test,length=look_back, batch_size=1)
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25
1029/1029 [==============================] - 7s 6ms/step - loss: 289183.6794
Epoch 2/25
1029/1029 [==============================] - 7s 6ms/step - loss: 6157.7021
Epoch 3/25
1029/1029 [==============================] - 6s 6ms/step - loss: 5689.4468
Epoch 4/25
1029/1029 [==============================] - 6s 6ms/step - loss: 4634.6634
Epoch 5/25
1029/1029 [==============================] - 7s 6ms/step - loss: 4044.8720
Epoch 6/25
1029/1029 [==============================] - 6s 6ms/step - loss: 3501.0008
Epoch 7/25
1029/1029 [==============================] - 6s 6ms/step - loss: 3093.6456
Epoch 8/25
1029/1029 [==============================] - 7s 6ms/step - loss: 2833.3519
Epoch 9/25
1029/1029 [==============================] - 6s 6ms/step - loss: 2656.1541
Epoch 10/25
1029/1029 [==============================] - 6s 6ms/step - loss: 2389.5288
Epoch 11/25
1029/1029 [==============================] - 6s 6ms/step - loss: 2246.3996
Epoch 12/25
1029/1029 [===========================

In [37]:
prediction = model.predict(test_generator)

ElectricityConsumption_train = ElectricityConsumption_train.reshape((-1))
ElectricityConsumption_test = ElectricityConsumption_test.reshape((-1))
prediction = prediction.reshape((-1))
import plotly.graph_objects as go


trace1 = go.Scatter(
    x = date_train,
    y = ElectricityConsumption_train,
    mode = 'lines',
    name = 'Train'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = ElectricityConsumption_test,
    mode='lines',
    name = 'Test'
)
layout = go.Layout(
    title = "ElectricityConsumption",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "ElectricityConsumption"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()